In [1]:
import torch
import matplotlib.pyplot as plt
import torch_dct as dct
from lib.tools import *
from lib.decomposition import *
from lib.jpeg import *
from lib.sketchml import *
from lib.regression import *
import os
import pandas as pd

In [2]:
def get_measurements(tensor):
    # tensor_rank.append(calculate_cp_rank(tensor.numpy(),max_rank=10,tol=1e-4))
    tensor_rank= calculate_slice_avg_rank(tensor)
    tensor_sparsity=calculate_sparsity(tensor.numpy())
    tensor_pictoriality=get_tensor_pictoriality(tensor)
    tensor_regularity=get_tensor_regularity(tensor)
    tensor_decomposability= get_tensor_decomposability(tensor)
    # print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank,tensor_sparsity*1.5,1-tensor_pictoriality/(tensor.numel()),1-tensor_regularity/(tensor.numel()),1-tensor_decomposability/(tensor.numel()*4)))
    print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank,tensor_sparsity,tensor_pictoriality,tensor_regularity,tensor_decomposability))
    # print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank,tensor_sparsity,tensor_pictoriality,tensor_regularity,tensor_decomposability))

    ## Jpeg compression
    print("Doing JPEG compression")
    normalize_base, scale,zero_point, encoded_data, compressed_size_jpeg = compressor_jpeg_gray(tensor, 100)
    reconstructed_tensor_jpeg = decompressor_jpeg_gray(tensor.shape,normalize_base, scale,zero_point, encoded_data )
    ratio_jpeg = tensor.numel()*4/compressed_size_jpeg
    snr_jpeg = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    mse_jpeg = calculate_mse(tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_jpeg,snr_jpeg,mse_jpeg))

    print("Doing Decomposition compression")
    # factors, compressed_size_decomposition = compressor_decomposition(tensor, tensor_rank[-1])
    # reconstructed_tensor_decomposition = decompressor_decomposition(factors)
    factors, compressed_size_decomposition = compressor_decomposition_slice(tensor)
    reconstructed_tensor_decomposition = decompressor_decomposition_slice(tensor.shape,factors)
    snr_decomposition = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    mse_decomposition = calculate_mse(tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    ratio_decomposition= tensor.numel()*4/compressed_size_decomposition
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_decomposition,snr_decomposition,mse_decomposition))

    print("Doing Regression compression")
    factors, x_pos, x_neg, compressed_size_regression = compressor_regression(tensor, 3)
    reconstructed_tensor_regression = decompressor_regression(tensor.shape, factors, x_pos,x_neg )
    snr_regression = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_regression.numpy())
    mse_regression = calculate_mse(tensor.numpy(), reconstructed_tensor_regression.numpy())
    ratio_regression = tensor.numel()*4 / compressed_size_regression
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_regression,snr_regression,mse_regression))

    print("Doing SketchML compression")
    minmaxsketch,bucket_means, keys, compressed_size_sketchml = compressor_sketchml(tensor,256,0.3,3)
    reconstructed_tensor_sketchml = decompressor_sketchml(tensor.shape, minmaxsketch,bucket_means, keys)
    snr_sketchml = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    mse_sketchml = calculate_mse(tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    ratio_sketchml = tensor.numel() *4 / compressed_size_sketchml
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_sketchml,snr_sketchml,mse_sketchml))


## Low Rank

In [3]:
tensor_shape = (3,128,128)
tensor_size  = 128*128*3
rank = 3

tensor = torch.zeros(tensor_shape)
for i in range(tensor_shape[0]):
    factor_1 = torch.randn(tensor_shape[1], rank)
    factor_2 = torch.randn(tensor_shape[2], rank)
    tensor_slice = torch.einsum('ir,jr->ij', factor_1, factor_2)
    tensor[i] = tensor_slice

In [4]:
get_measurements(tensor)

Rank: 3.000000, Sparsity: 0.000000, Picoriality: 0.184372, Regularity: 0.157485, Decomposability: 0.953125
Doing JPEG compression
Ratio: 1.668616, SNR: 32.919881, MSE: 0.001653
Doing Decomposition compression
Ratio: 21.333333, SNR: 131.031761, MSE: 0.000000
Doing Regression compression
Ratio: 15.953262, SNR: 3.472039, MSE: 1.455232
Doing SketchML compression
Ratio: 4.150563, SNR: 1.744457, MSE: 2.166159


## Image-like

In [5]:
import cv2
import torchvision.transforms as transforms
image = cv2.imread("./dataset/jpeg/jpeg422jfif.jpg")
image = cv2.resize(image,(128,128),interpolation = cv2.INTER_AREA)
tensor = torch.from_numpy(image)
value_mask = abs(tensor)>10
tensor = tensor* value_mask
tensor = tensor.to(torch.float32)/256
# tensor = torch.load("./test_tensors/layer2/tensor_l2_34.pt")
# tensor= tensor[0]

In [6]:
dct_tensor = dct.dct_2d(tensor.reshape(tensor.shape[0], tensor.shape[1]*tensor.shape[2]))

In [7]:
get_measurements(tensor)

Rank: 3.000000, Sparsity: 0.014567, Picoriality: 0.665067, Regularity: 0.619908, Decomposability: -0.023438
Doing JPEG compression
Ratio: 5.130421, SNR: 13.042058, MSE: 0.006970
Doing Decomposition compression
Ratio: 0.977099, SNR: 132.847872, MSE: 0.000000
Doing Regression compression
Ratio: 14.222222, SNR: 8.516518, MSE: 0.019761
Doing SketchML compression
Ratio: 4.213179, SNR: nan, MSE: nan


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/sketchml.py:185: RuntimeWarning: Mean of empty slice.
  bucket_means[k] = values[bucket_mask].mean()
/home/rex/archiconda3/envs/pytorch/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


## Regular distributed

In [8]:
import random
random.seed(5)
tensor = torch.zeros([128,128*3])

for i in range(128):
    a = random.random()
    b = random.random()*2
    c = random.random()
    for j in range(128*3):
        index = j /(128*3)
        if random.random()>0.5:
            tensor[i][j] = a*index -b*index*index+c*index*index*index+ random.random()/10
            # tensor[i][j] = a*index + random.random()/10
tensor = tensor.reshape([128,128,3])

In [9]:
get_measurements(tensor)

Rank: 3.000000, Sparsity: 0.503194, Picoriality: 0.393564, Regularity: 0.646187, Decomposability: -0.023438
Doing JPEG compression
Ratio: 4.773545, SNR: 34.063463, MSE: 0.000024
Doing Decomposition compression
Ratio: 0.977099, SNR: 137.717218, MSE: 0.000000
Doing Regression compression
Ratio: 14.222222, SNR: 21.112418, MSE: 0.000479
Doing SketchML compression
Ratio: 7.840172, SNR: 2.529085, MSE: 0.034539


## Random tensor

In [10]:
import random
random.seed(5)
tensor = torch.zeros([128,128*3])
for i in range(128):
    for j in range(128*3):
        if random.random()>0.7:
            tensor[i][j] = random.gauss(1, 0.3)
tensor = tensor.reshape([128,128,3])

In [11]:
get_measurements(tensor)

Rank: 3.000000, Sparsity: 0.701274, Picoriality: 0.333151, Regularity: 0.478078, Decomposability: -0.023438
Doing JPEG compression
Ratio: 2.823327, SNR: 17.425761, MSE: 0.005877
Doing Decomposition compression
Ratio: 0.977099, SNR: 136.207123, MSE: 0.000000
Doing Regression compression
Ratio: 14.222222, SNR: 10.919476, MSE: 0.026290
Doing SketchML compression
Ratio: 12.033786, SNR: 12.059802, MSE: 0.020219
